In [232]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore


dataset_path = "cancer_dataset"

In [233]:
# Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
    )
training_set = train_datagen.flow_from_directory(
    f'{dataset_path}/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

Found 612 images belonging to 4 classes.


In [234]:
# Validition
val_datagen = ImageDataGenerator(rescale=1./255)
val_set = val_datagen.flow_from_directory(
    f'{dataset_path}/valid',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

Found 173 images belonging to 4 classes.


In [235]:
# Initialising the CNN|
cnn = tf.keras.models.Sequential()

In [236]:
# Step1 - Convolution
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu',
    input_shape=[64, 64, 3]))

c:\Users\Matrix Store\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [237]:
# Step2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [238]:
# cnn.add(tf.keras.layers.Dropout(0.5))

In [239]:
# 2nd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [240]:
# cnn.add(tf.keras.layers.Dropout(0.5))

In [241]:
# 3rd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [242]:
# Step3 - Fallenting
cnn.add(tf.keras.layers.Flatten())

In [243]:
# Step4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [244]:
# Step5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

In [245]:
# Compile the CNN
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [246]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# افترض إن عندك أسماء الكلاسات:
class_names = list(training_set.class_indices.keys())

# خد كل الليبلز من generator
labels = training_set.classes  # أرقام الكلاسات

# احسب class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# حولهم لقاموس
class_weights = dict(enumerate(class_weights))


In [247]:
# Training the CNN on the Training set
# and evaluation it on the Test set
cnn.fit(x = training_set, validation_data=val_set, epochs=30)

Epoch 1/30


c:\Users\Matrix Store\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.2838 - loss: 1.3595 - val_accuracy: 0.3642 - val_loss: 1.3105
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 177ms/step - accuracy: 0.3977 - loss: 1.2541 - val_accuracy: 0.3295 - val_loss: 1.2852
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 209ms/step - accuracy: 0.4630 - loss: 1.1495 - val_accuracy: 0.3815 - val_loss: 1.1741
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.4942 - loss: 1.0800 - val_accuracy: 0.5145 - val_loss: 1.0566
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 228ms/step - accuracy: 0.5862 - loss: 1.0109 - val_accuracy: 0.4855 - val_loss: 1.0620
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step - accuracy: 0.6531 - loss: 0.7994 - val_accuracy: 0.5954 - val_loss: 0.8527
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 158ms/step - accuracy: 0.6437 - loss: 0.8158 - val_accuracy: 0.5491 - val_loss: 1.1377
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - accuracy: 0.6017 - loss: 0.8372 - val_accuracy: 0.5549 - val_

In [248]:
# Making a Single prediction
import numpy as np
from keras.preprocessing import image
import os
import random

test_folders = ['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']
random_folder = random.choice(test_folders)
folder_path = os.path.join(f'{dataset_path}/test', random_folder)
images = os.listdir(folder_path)
random_image = random.choice(images)
image_path = os.path.join(folder_path, random_image)

print(image_path)

test_image = image.load_img(image_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)

print(result)
print(training_set.class_indices)

if result[0][0] == 0:
    prediction = 'Adenocarcinoma'
elif result[0][0] == 1:
    prediction = 'Large Cell Carcinoma'
elif result[0][0] == 2:
    prediction = 'Normal'
elif result[0][0] == 3:
    prediction = 'Squanmous Cell Carcinoma'

print(prediction)

cancer_dataset/test\adenocarcinoma\000149 (7).png


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
[[0. 0. 1. 0.]]
{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1, 'normal': 2, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}
Adenocarcinoma
